### Universal dependencies with __[Stanza](https://stanfordnlp.github.io/stanza/#getting-started)__ and __[spacy_conll](https://spacy.io/universe/project/spacy-conll)__

#### Побудова дерев залежностей текстів новин і збереження їх у форматі conll для подальшої обробки моделлю, що визначає тональність

In [2]:
import pandas as pd
import os
import re
import razdel
import stanza
import six
from spacy_conll import init_parser

In [2]:
news_filepath = '/Users/oksana/Dev/data/may.csv'
news = pd.read_csv(news_filepath)

news['title'] = news.title.str.replace(r'\n', ' ')
news['all_text'] = news.title.str.cat(news.subtitle, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.cat(news.text, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.strip()

news = news.drop(columns=['title', 'subtitle', 'text'])

news['all_text'] = news['all_text'].str.replace('не очень', 'неочень')
news['all_text'] = news['all_text'].str.replace('не дуже', 'недуже')

news.all_text = news.all_text.str.replace(r'^Редактор Цензор\.НЕТ\n', '', flags=re.M)
news.all_text = news.all_text.str.replace(r'Цензор\.НЕТ', 'Цензор')
news.all_text = news.all_text.str.replace(r'Еспресо\.Захід', 'Еспресо Захід')

/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [2]:
stanza.download('uk')
stanza.download('ru', package='gsd', processors='tokenize,pos,lemma,depparse')

nlp = stanza.Pipeline('uk', processors='tokenize,lemma')

nlp_uk = init_parser(
        "stanza",
        "uk", 
        include_headers=False,
        parser_opts = {'processors': 'tokenize,pos,lemma,depparse'}
    )

nlp_ru = init_parser(
        "stanza",
        "ru", 
        include_headers=False,
        parser_opts = {'processors': 'tokenize,pos,lemma,depparse'}
    )

2021-10-08 13:15:37 INFO: Downloading default packages for language: uk (Ukrainian)...
2021-10-08 13:15:38 INFO: File exists: /Users/oksana/stanza_resources/uk/default.zip.
2021-10-08 13:15:43 INFO: Finished downloading models and saved to /Users/oksana/stanza_resources.
2021-10-08 13:15:43 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| pretrain  | gsd     |

2021-10-08 13:15:43 INFO: File exists: /Users/oksana/stanza_resources/ru/tokenize/gsd.pt.
2021-10-08 13:15:43 INFO: File exists: /Users/oksana/stanza_resources/ru/pos/gsd.pt.
2021-10-08 13:15:43 INFO: File exists: /Users/oksana/stanza_resources/ru/lemma/gsd.pt.
2021-10-08 13:15:44 INFO: File exists: /Users/oksana/stanza_resources/ru/depparse/gsd.pt.
2021-10-08 13:15:44 INFO: File exists: /Users/oksana/stanza_resources/ru/pretrain/gsd.pt.
2021-10-08 13:15:44 INF

In [19]:
s1="Такий прогноз озвучив в ефірі \"Україна 24\" президент Всеукраїнської асоціації компаній з міжнародного працевлаштування Василь Воскобойник"
doc = nlp_uk(s1)
conll = doc._.conll_str
print(conll)

1	Такий	такий	DET	Pd--m-sna	_	2	det	_	_
2	прогноз	прогноз	NOUN	Ncmsnn	_	3	nsubj	_	_
3	озвучив	озвучити	VERB	Vmeis-sm	_	0	root	_	_
4	в	в	ADP	Spsl	_	5	case	_	_
5	ефірі	ефір	NOUN	Ncmsln	_	3	obl	_	_
6	"Україна	"україна	NOUN	Ncfsny	_	3	nsubj	_	_
7	24"	24"	PUNCT	U	_	8	discourse	_	_
8	президент	президент	NOUN	Ncmsny	_	6	appos	_	_
9	Всеукраїнської	всеукраїнський	ADJ	Ao-fsgf	_	10	amod	_	_
10	асоціації	асоціація	NOUN	Ncfsgn	_	8	nmod	_	_
11	компаній	компанія	NOUN	Ncfpgn	_	10	nmod	_	_
12	з	з	ADP	Spsg	_	14	case	_	_
13	міжнародного	міжнародний	ADJ	Ao-nsgf	_	14	amod	_	_
14	працевлаштування	працевлаштування	NOUN	Ncnsgn	_	8	nmod	_	_
15	Василь	Василь	PROPN	Npmsny	_	6	flat:title	_	_
16	Воскобойник	Воскобойник	PROPN	Npmsny	_	15	flat:name	_	SpaceAfter=No



In [7]:
news_uk = news[news.mentions.notna()&(news.language=='uk')]
news_ru = news[news.mentions.notna()&(news.language=='ru')]
news.shape

(27434, 17)

#### Кожну новину, у якій є згадки політиків, обробляємо і зберігаємо у conll файл з ім'ям, що відповідає індексу новини

In [9]:
def save_to_conllu(doc, news_id, out_dir='/Users/oksana/Dev/Politicians_in_media/may_conllu/conllu_uk/'):
    out_file = out_dir + "{}.conll".format(news_id)
    for sent_idx, sent in enumerate(doc.sents, 1):
        header = ['### ', sent_idx, news_id]
        pd.DataFrame([header]).to_csv(out_file, sep='\t', index=False, header=None, mode='a')
        sent._.conll_pd.to_csv(out_file, index=False, sep="\t", encoding='utf-8', mode='a', header=None)
        with open(out_file, 'a') as f:
            f.write('\n')

In [ ]:
news_part = None

**Обробка займає багато часу, тому запускаємо по черзі на частинах масиву (паралельно іншу частину можна обробляти в Colab)**

In [15]:
def process_news(news, nlp=nlp_uk, start=0, finish=1000, step=100, out_dir='/Users/oksana/Dev/Politicians_in_media/may_conllu/conllu_uk/'):
    for k in range(start, finish, step):
        news_part = news.iloc[k:k + step].copy()
        news_part['docs'] = news_part.tokenized.apply(nlp)
        news_part.apply(lambda row: save_to_conllu(row.docs, row['id'], out_dir), axis=1)
        print(k+step)

In [ ]:
%%time
process_news(news_uk, nlp_uk, start=12453, finish=13000, step=100)

In [ ]:
%%time
process_news(news_ru, nlp_ru, start=0, finish=len(news_ru), step=200, out_dir='/Users/oksana/Dev/Politicians_in_media/may_conllu/conllu_ru/')

### Sentiment analysis with __[UUUSA](https://github.com/aghie/uuusa)__   ( __[article](https://arxiv.org/abs/1606.05545)__,  __[manual](http://grupolys.org/software/UUUSA/uuusa-user-manual.pdf)__ )

**Далі отримані conll файл подаємо на вхід моделі UUUSA (у файлі samulan-0.1.0.jar)**
Вказуємо для неї параметри:
 - папку із тональними словниками для української/російської мови
 - файл з конфігураціями
 - сам conll файл
 - властивості моделі
 - verbose
 - вихідний файл, куди збережеться результат

In [ ]:
java -Dfile.encoding=UTF-8 -jar -Xmx2g samulan-0.1.0.jar \ 
-s UkSentiData \
-r configuration_uk.xml \
-c parsed.conll \
-p samulan.properties \
-v true
-o output.txt

In [ ]:
for file in *.conll; do java -Dfile.encoding=UTF-8 -jar -Xmx2g ../samulan-0.1.1.jar \
-s ../UkSentiData \
-r ../configuration_uk.xml \
-c $file \
-p ../samulan.properties\
-sc so \
-o ../output/$file; done

**Вихідний файл має такий вигляд (перший стовпчик - тональність речення)**

    1.0		Каменських назвала свій головний привід для радості , і це не Потап : """" Згодні ? """"	 [The analysis took: 0.060065182 seg.] [Accumulated time is: 0.060065182]
    3.0		Популярна українська співачка Настя Каменських , яка минулого року вийшла заміж за свого колегу Олексія Потапенка , часто радує публіку яскравими постами в своєму Instagram .	 [The analysis took: 0.001861094 seg.] [Accumulated time is: 0.061926276]
    1.0		На карантині у зірки з'явилося більше часу для спілкування з фанатами , чим вона активно користується .	 [The analysis took: 0.001455214 seg.] [Accumulated time is: 0.06338149]
    0.0		Читайте Знай в Google News !	 [The analysis took: 7.0612E-4 seg.] [Accumulated time is: 0.06408761
    2.0		Нещодавно дружина Потапа розмістила свіже фото , на якому позує біля пальми в легкому жіночному образі – жовтих шортах , білій сорочці та верху від купальника теж білого кольору .	 [The analysis took: 0.002728221 seg.] [Accumulated time is: 0.066815831]
    1.0		У підписі Каменських зазначила , що п'ятниця вже є приводом для радості .	 [The analysis took: 0.002728221 seg.] [Accumulated time is: 0.066815831]


#### Зчитуємо потім ці файли з тональністю, зберігаємо результат - сумарну тональність новини (sentiment) і список (точніше рядок чисел, що розділені через ;) тональностей окремих її речень (sent_list)

In [ ]:
sentiment_files = os.listdir(sentiment_dir)
res = []
for file in sentiment_files:
    s = pd.read_csv(file, sep='\t', usecols=[0], header=None)[0]
    res.append((file.strip('.conll'), s.sum(), s.astype(str).str.cat(sep=';')))
df = pd.DataFrame(res, columns=['id', 'sentiment', 'sent_list'])
df.id = df.id.astype('int64')